In [2]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
import numpy as np

In [2]:
from pycuda.reduction import ReductionKernel

# Calcule l'erreur de la matrice
error_kernel = ReductionKernel(dtype_out=np.float32, neutral="0",
                     reduce_expr="a+b", map_expr="(x[i]-y[i])*(x[i]-y[i])",
                     arguments= "float *x, float *y")
# NE FONCTIONNE PAS POUR L'INSTANT, on utilise un ElementwistKernel

### Petit benchmark

Crois-les, ils sont longs à relancer et ça met longtemps

Pour vérifier la mémoire utilisée sur la cg, nvidia-smi

A priori ElementwiseKernel est bien le plus rapide, en plus la mémoire est mieux gérée qu'en tentant les opérations algébriques directement

In [3]:
from pycuda.elementwise import ElementwiseKernel

error_kernel = ElementwiseKernel("const float *x, const float *y, float *z",
                                "z[i] = (x[i] - y[i])*(x[i] - y[i])",
                                "error_kernel")

In [34]:
a = np.random.randn(10000, 10000).astype(np.float32)
b = np.random.randn(10000, 10000).astype(np.float32)
a_gpu = gpuarray.to_gpu(a)
b_gpu = gpuarray.to_gpu(b)
diff_matrix = gpuarray.empty_like(a_gpu)

In [35]:
%%timeit
((a_gpu - b_gpu)*(a_gpu - b_gpu)).get() # En plus soucis de mémoire

MemoryError: cuMemAlloc failed: out of memory

In [39]:
%%timeit
error_kernel(a_gpu , b_gpu, diff_matrix) # Quand le code est-il exécuté ? Seulement quand on get le truc ?

27.3 ms ± 482 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [40]:
%%timeit
k = (a-b)**2

373 ms ± 368 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit
k = diff_matrix.get()

91.6 ms ± 544 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
# Calcul de la pénalité pour chaque matrice pour éviter l'overfitting
a = np.arange(8).reshape(4,2)
a_gpu = gpuarray.to_gpu(a)
b = np.arange(1, 10).reshape(3,3)
b_gpu = gpuarray.to_gpu(b)

In [4]:
# Met tous les élements de la matrice au carré
square_kernel = ElementwiseKernel("const float *x", "float *y",
                                  "y[i] = x[i]*x[i]",
                                  "square_kernel")

# Somme ligne par ligne
line_sum_mod = SourceModule("""
                        #include <stdio.h>
                        
                        __global__ void sumcolumns (const long *a, long *b)
                        {
                            int ncols = 2;
                            int idx_start = threadIdx.x;
                            long sum = 0;
                            for (int idx = ncols*idx_start; idx < ncols*idx_start + ncols; idx++)
                                sum += a[idx];
                            b[idx_start] = sum;
                        }""")


# Somme colonne par colonne
column_sum_mod = SourceModule("""
                        #include <stdio.h>
                        
                        __global__ void sumcolumns (const long *a, long *b)
                        {
                            int nlines = 4;
                            int ncols = 2; 
                            int idx_start = threadIdx.x + threadIdx.y*ncols;
                            long sum = 0;
                            for (int idx = 0; idx <  nlines; idx++)
                                sum += a[idx_start + ncols*idx];
                            b[idx_start] = sum;
                        }""")



lsummod = line_sum_mod.get_function("sumcolumns")
csummod = column_sum_mod.get_function("sumcolumns")

In [8]:
out_sum_columns = gpuarray.zeros((1,2), dtype=int)
csummod(a_gpu, out_sum_columns, block=(2,2,1)) # Pourquoi les sommes pour (4,1,1) et (2,2,1) ne sont pas équivalentes ?

In [9]:
out_sum_columns.get()

array([[12, 16]])

In [10]:
out_sum_lines = gpuarray.zeros((1,4), dtype=int)
lsummod(a_gpu, out_sum_lines, block=(32,32,1))

In [11]:
out_sum_lines.get()

array([[ 1,  5,  9, 13]])

In [12]:
a_gpu.get()

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

In [ ]:
def cost_function(real_matrix, predicted_matrix, l_u, l_v, p, q):
    # Faire la matrice des différences au carré,
    # Calcule le coût de p
    # Calcule celui de q
    return (real_matrix - predicted_matrix)**2 + l_u*norm(p) + l_v * norm(q)

# Multiplication des matrices p et q

Nécessaire pour obtenir l'erreur totale

In [187]:
from pycuda.tools import DeviceData

In [188]:
# Multiplication ligne par ligne (ou colonne par colonne)
# On peut cacher les données de la ligne
# Que faire quand il y en a trop ?

1024

1024

# Création de la matrice principale

On va se concentrer sur un modèle qui fonctionne, donc on crée une matrice où l'on sait qu'il existe une solution.
Pour cela, on crée une des matrices originales P_o et Q_o que l'on multiplie.

On note que contrairement à une véritable dataset, la matrice sera ici très dense avec chaque cellule contenant une valeur. Dans la vraie vie (Notes sur Netflix, Amazon, etc...), les matrices comprennent énormément de NaNs, qu'on ne retrouvera pas ici


In [10]:
# Générer deux "vraies" matrices P et Q, faire le produit
p_o = np.random.randn(100, 1000) # Generate from uniform(0, 1)
q_o = np.random.randn(1000, 100)

On en profite pour faire un benchmark de la vitesse de mutliplication de matrice par numpy et par cuda
On remarque d'ailleurs que la différence de facteur augmente au fur et à mesure qu'on augmente la taille de la matrice

In [17]:

%timeit np.matmul(q_o, p_o)

3.05 ms ± 320 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [91]:
line_matmul = SourceModule("""
    #include <stdio.h>
    
    __global__ void prodbyline (const float *p, const float *q, float *r, const int nq, const int np, const int ncom)
    {
    printf("__NEW__");
    // TODO : cache the line
    const int startp = (threadIdx.x + threadIdx.y + threadIdx.z)* ncom;
    const int startq = threadIdx.x + threadIdx.y + threadIdx.z; 
    const int startr = (threadIdx.x + threadIdx.y + threadIdx.z) * nq;
    printf("P : %d, Q : %d, R : %d\\n", startp, startq, startr);
    
    for (int idcell = startr; idcell < startr + nq; idcell++)
     {
        float sumcell = 0;
        for(int idy = startq, idx = startp;
            idy <= startq + nq*ncom;
            idy += nq, idx++)
            sumcell += p[idx] * q[idy];
            printf("For id %d, value is %d\\n", idcell, sumcell);
            r[idcell] = sumcell;
            }
        }
            """)
#Todo : corriger ça

In [9]:
a = np.arange(1, 11).reshape(2,5).astype(np.float32)
b = a.T.copy()
a_gpu = gpuarray.to_gpu(a)
b_gpu = gpuarray.to_gpu(b)



In [93]:
matmul = line_matmul.get_function("prodbyline")

In [94]:
out_mat = gpuarray.zeros((2,2), dtype=int)
matmul(a_gpu, b_gpu, out_mat, np.int32(5), np.int32(5), np.int32(2), block=(4,1,1))
out_mat.get()

array([[9223372034707292159, 9223372034707292159],
       [9223372034707292159, 9223372034707292159]])

### Que faire quand + de lignes que de threads possible ?
- Modulo nb total de threads
- "maillage" en python

# Utilisation du grid

In [6]:
gridmod = SourceModule("""
    #include <stdio.h>
    
    __global__ void reperage (float *a)
    {
    printf("__NEW__");
    const int idx = threadIdx.x + 5*threadIdx.y;
    a[0] = idx;
    printf("Block is (%d, %d), thread is (%d, %d):\\n", blockIdx.x, blockIdx.y, threadIdx.x, threadIdx.y);
    }""")

In [7]:
tryprint = gridmod.get_function('reperage')

In [26]:
tryprint(a_gpu, block=(2,1,1), grid=(1,1))

In [27]:
a_gpu.get() # ??? Seulement executé quand on appelle get ?

array([[  1.,   2.,   3.,   4.,   5.],
       [  6.,   7.,   8.,   9.,  10.]], dtype=float32)

# Dimension d'un block
Je ne comprends pas exactement quel est l'intérêt d'écrire un block en plus d'une dimension. Est-ce que la vitesse est meilleure dans certains cas pour (32, 16, 2) que (1024,1,1) ?
A tester